## Notebook 02: Predictive Modeling & Evaluation

Objective: To develop, calibrate, and evaluate a Seasonal Autoregressive Integrated Moving Average (SARIMA) model against established persistence baselines to define the limits of short-term thermal forecasting.
1. Model Identification & Environmental DNA

In this phase, we move from observation to estimation. Our goal is to translate the physical patterns found in Notebook 01 (Daily Waves and Thermal Inertia) into mathematical parameters (p,d,q,P,D,Q,s).

Intentional Logic:

- Temporal Consistency: We enforce a strict 5-minute frequency to ensure the model understands the "distance" between observations.

- Autocorrelation Analysis: We use ACF and PACF plots to determine how many past "pings" directly influence the future temperature.

- Benchmark Alignment: Every result here is directly compared to the 0.0219∘C, 0.0351∘C, and 0.0549∘C benchmarks we set earli

We start by loading the processed data. We must ensure the Timestamp is once again recognized as the index and that the frequency is explicitly set to 5min. Without an explicit frequency, the SARIMA math will fail.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- LOGIC: LOADING CHECKPOINT DATA ---
df = pd.read_csv('../data/processed/cleaned_iot_data.csv', index_col='Timestamp', parse_dates=True)

# Explicitly set the frequency (Essential for SARIMA logic)
df = df.asfreq('5min')

print("-" * 30)
print("DATASET READY FOR MODELING")
print("-" * 30)
print(f"Total Observations: {len(df)}")
print(f"Sampling Frequency: {df.index.freqstr}")
print("-" * 30)
df.head()

2. Identifying Model Parameters (ACF & PACF)

Intent: We need to figure out our p,d,q values.

- ACF (Autocorrelation): Shows how much the current temp is correlated with its own past.

- PACF (Partial Autocorrelation): Helps us find the "direct" relationship by stripping away intermediate correlations.

In [ ]:
# --- LOGIC: SIGNAL FINGERPRINTING ---
# We look at the first 50 lags (approx 4 hours of data)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

plot_acf(df['Temp_C'], lags=50, ax=ax1)
plot_pacf(df['Temp_C'], lags=50, ax=ax2, method='ywm')

plt.tight_layout()
plt.show()

 3. . Multi-Horizon Stress Testing

**The "Breaking Point" Logic**
To evaluate the stability of the SARIMA model, we perform a comparative analysis across three forecasting horizons: 10, 30, and 60 minutes. 

* **Short-Range (10m):** Tests the model's ability to handle high-frequency sensor jitter.
* **Mid-Range (30m):** Tests the transition between immediate inertia and the start of thermal trends.
* **Long-Range (60m):** Tests the model's reliance on the seasonal daily cycle.

By comparing the SARIMA MAE against the Persistence MAE at each step, we can identify the specific point where environmental "noise" or sensor limitations cause the model to lose its predictive advantage.

(The 5/2 Day Split & Model Training)

We are training the model on the first 1440 observations (5 days at 12 pings/hour) and holding out the rest for testing.

In [ ]:
# --- LOGIC: REFINED MULTIVARIATE TRAINING ---
train_size = 1440
train_data = df['Temp_C'].iloc[:train_size]
train_exog = df['Humidity'].iloc[:train_size]

test_data = df['Temp_C'].iloc[train_size:]
test_exog = df['Humidity'].iloc[train_size:]

print(f"Training Refined ARIMAX (1,0,0) on 5 days data...")

# SWITCH: Using d=0 to prevent 'noise amplification' 
model_cfg = SARIMAX(train_data, 
                    exog=train_exog,
                    order=(1, 0, 0), 
                    enforce_stationarity=False,
                    enforce_invertibility=False,
                    initialization='approximate_diffuse') 

arimax_fit = model_cfg.fit(disp=False, low_memory=True) 

print("Refined ARIMAX Model trained successfully.")

(Multi-Horizon Validation & Benchmarking)

This block generates the forecasts for your specific horizons and calculates the MAE, RMSE, and R2 against the Persistence Baseline.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

horizons = {'10-Min': 2, '30-Min': 6, '60-Min': 12} 
results_table = []
plot_data = {label: {'act': [], 'pred': []} for label in horizons}

# Define evaluation window (24 hours / 288 pings)
test_subset_temp = test_data.head(288)
test_subset_exog = test_exog.head(288)

print(" Starting Refined Rolling Evaluation...")

for label, steps in horizons.items():
    print(f"Analyzing {label} horizon...")
    
    for i in range(len(test_subset_temp) - steps):
        # LOGIC: Use a shorter tail (200) to keep the model reactive to 'Now'
        current_temp_history = pd.concat([train_data, test_subset_temp.iloc[:i]]).tail(200)
        current_exog_history = pd.concat([train_exog, test_subset_exog.iloc[:i]]).tail(200)
        
        future_exog = test_subset_exog.iloc[i : i + steps]
        
        # Fit Stable ARIMAX(1,0,0)
        model_roll = SARIMAX(current_temp_history, 
                             exog=current_exog_history, 
                             order=(1, 0, 0), 
                             enforce_stationarity=False)
        model_fit = model_roll.fit(disp=False)
        
        # Forecast
        forecast = model_fit.get_forecast(steps=steps, exog=future_exog)
        
        plot_data[label]['pred'].append(forecast.predicted_mean.iloc[-1])
        plot_data[label]['act'].append(test_subset_temp.iloc[i + steps])
        
    # Metrics
    y_true = plot_data[label]['act']
    y_pred = plot_data[label]['pred']
    y_persist = test_subset_temp.iloc[:len(y_true)] # Naive Baseline
    
    mae = mean_absolute_error(y_true, y_pred)
    p_mae = mean_absolute_error(y_true, y_persist)
    
    results_table.append({
        'Horizon': label,
        'Persist MAE': round(p_mae, 4),
        'Refined ARIMAX MAE': round(mae, 4),
        'Improvement': f"{((p_mae - mae) / p_mae * 100):.2f}%"
    })

# Output Results
final_results_df = pd.DataFrame(results_table)
print("\n" + "="*65)
print("FIXED MULTIVARIATE PERFORMANCE: STABLE ARIMAX")
print("="*65)
print(final_results_df.to_string(index=False))

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(plot_data['10-Min']['act'][:100], label='Actual', color='black', alpha=0.7)
plt.plot(plot_data['10-Min']['pred'][:100], label='ARIMA 10m', color='orange', linestyle='--')
plt.title('10-Minute Stress Test: One-Step Tracking')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(plot_data['30-Min']['act'][:100], label='Actual', color='black', alpha=0.7)
plt.plot(plot_data['30-Min']['pred'][:100], label='ARIMA 30m', color='blue', linestyle='--')
plt.title('30-Minute Stress Test: Trend Response')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(plot_data['60-Min']['act'][:100], label='Actual', color='black', alpha=0.7)
plt.plot(plot_data['60-Min']['pred'][:100], label='ARIMA 60m', color='green', linestyle='--')
plt.title('60-Minute Stress Test: Long-Range Stability')
plt.legend()
plt.show()

In [ ]:
# Final Plot: Accuracy vs Sensor Tolerance
plt.figure(figsize=(10, 6))
horizons_list = ['10-Min', '30-Min', '60-Min']
errors = [0.0317, 0.0565, 0.0938] # Your Refined MAEs

plt.bar(horizons_list, errors, color='skyblue', label='Refined ARIMA MAE')
plt.axhline(y=0.5, color='red', linestyle='--', label='DHT22 Tolerance (±0.5°C)')

plt.title('Predictive Reliability: Forecast Error vs. Sensor Tolerance')
plt.ylabel('Mean Absolute Error (°C)')
plt.ylim(0, 0.6) # Scale to show the 0.5 threshold clearly
plt.legend()
plt.text(0, 0.52, 'Reliability Zone (Acceptable)', color='green', fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.show()